## Tweepy API Set-Up

In [0]:
!pip install tweepy

In [0]:
import tweepy
import pandas as pd

In [0]:
ckey="37PWOOA3Ufj4yFJAvn7kH4ocX"
csecret="cUMxyTKUuiYw5tEGL0NjQ4utCoXmzTQG2H7leZxUqmYthgsrQK"
atoken="955975812-MIu6rEdMBgTwCcxBz6mTDZrzsPk9W36nCxnOaFpH"
asecret="Lra3hBdFBg8rMjd1vmjyrC00eBa90Tasqa9njx89plx08"

auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Experimenting

In [0]:
user = api.get_user('jdragsky')

print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

jdragsky
862
ExaltedFuneral
Spwack
NightinGem
MnemonicRpg
wanwuslibrary
urwronglol
sublimemarch
jerskann
audreyotherway
King_Crackers
thejessleigh
quinnntastic_
bigdipperstan
Drooling_Demon
codexnoirmatic
DakotaSkyBloom
Rathayibacter
LeviathanFiles
natashaence
onlavenderwings


In [0]:
#test_status = pd.DataFrame(api.get_status(1226512561832591360))
#print(test_status)
#https://twitter.com/SarahCullison/status/1226314461415342080

test_status = api.get_status(1226314461415342080)

print('EGO SOURCE TWEET INFO')
print('favorite count: ' + str(test_status._json['favorite_count']))
print('retweet count: '+ str(test_status._json['retweet_count']))
print('is this a retweet? ' + str(test_status.retweeted))
print("")
print('SOURCE USER INFO')
print('favorites count: ' + str(test_status._json['user']['favourites_count']))
print('followers count: ' + str(test_status._json['user']['followers_count']))
print('friends count: ' + str(test_status._json['user']['friends_count']))
print('id: ' + str(test_status._json['user']['id']))
print('statuses count: ' + str(test_status._json['user']['statuses_count']))

EGO SOURCE TWEET INFO
favorite count: 330
retweet count: 98
is this a retweet? False

SOURCE USER INFO
favorites count: 252
followers count: 887
friends count: 1369
id: 723368133562527745
statuses count: 2257


## Edge List

In [0]:
!pip install flatten_json

import flatten_json

In [0]:
test_retweets = api.retweets(1226314461415342080)
test_retweeters = api.retweeters(1226314461415342080)

test = []

for i in range(0,len(test_retweets)):
  test.append(flatten_json.flatten_json(test_retweets[i]._json))

retweet_df = pd.io.json.json_normalize(test)
#list(testdf.columns)
#retweet_df

len(test_retweeters)

In [0]:
retweeter_names = []

for r in test_retweeters:
  retweeter_names.append(api.get_user(r)._json['screen_name'])

In [0]:
V1 = list()
V2 = list()

def checkFollow(sourceid, targetid):
  relation = api.show_friendship(source_screen_name=sourceid, 
                                 target_screen_name=targetid)
  if relation[0].following and not relation[1].following:
    V1.append(sourceid)
    V2.append(targetid)
  elif relation[1].following and not relation[0].following:
    V1.append(targetid)
    V2.append(sourceid)
  elif relation[0].following and relation[1].following:
    V1.append(targetid)
    V2.append(sourceid)
    V1.append(sourceid)
    V2.append(targetid)
  #return(relation[0].following, relation[1].following)

In [0]:
from itertools import combinations

cnt = 0

for l in list(combinations(retweeter_names, 2)):
  cnt += 1
  print(cnt)
  checkFollow(l[0], l[1])


In [0]:
reblog_edges = pd.DataFrame({"V1":V1,"V2":V2})

reblog_edges.to_csv("retweet_edges2.csv")

## Node List

In [0]:
V3 = set(reblog_edges.V1)

for v in reblog_edges.V2:
  V3.add(v)

V3 = list(V3)

retweet_nodes = pd.DataFrame({"names":list(set(V3))})

In [0]:
retweeter_ids = []

for v in V3:
  retweeter_ids.append(api.get_user(v)._json['id'])

retweet_nodes['uuid'] = retweeter_ids

In [0]:
favourites_count = list()
lang = list()
followers_count = list()
description = list()
friends_count = list()
listed_count = list()
statuses_count = list()

for i in retweet_nodes.uuid:
  favourites_count.append(api.get_user(i)._json['favourites_count'])
  lang.append(api.get_user(i)._json['lang'])
  followers_count.append(api.get_user(i)._json['followers_count'])
  description.append(api.get_user(i)._json['description'])
  friends_count.append(api.get_user(i)._json['friends_count'])
  listed_count.append(api.get_user(i)._json['listed_count'])
  statuses_count.append(api.get_user(i)._json['statuses_count'])

In [0]:
retweet_nodes['favourites_count'] = favourites_count
retweet_nodes['lang'] = lang
retweet_nodes['followers_count'] = followers_count
retweet_nodes['description'] = description
retweet_nodes['friends_count'] = friends_count
retweet_nodes['listed_count'] = listed_count
retweet_nodes['statuses_count'] = statuses_count

In [0]:
retweet_nodes.to_csv("retweet_nodes.csv")